In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from WindPy import w
w.start()
from function import *
from nav_research import NavResearch
from pathlib import Path
import os
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [108]:
data_path = r"data\旭诺CTA三号净值20250109.xlsx"
strategy = "量化CTA"
fund_name = "旭诺CTA三号"
benchmark_code = "NH0100.NHF"
benchmark_name = "南华商品指数"
threshold = -0.05

demo = NavResearch(data_path,strategy,fund_name,benchmark_code,benchmark_name,threshold)
df_nav, df_return, df_drawdown = demo.get_data()

In [109]:
data = df_nav[["date", "nav_adjusted"]]

In [110]:
nav_month_end = data.sort_values("date").groupby(data["date"].dt.to_period("M")).tail(1)
nav_month_end

,date,nav_adjusted
4,2019-06-28,0.9997
8,2019-07-26,0.9993
13,2019-08-30,0.9463
18,2019-09-30,0.9590
21,2019-10-25,0.9860
...,...,...
273,2024-09-30,3.2840
276,2024-10-25,3.3492
281,2024-11-29,3.6965
285,2024-12-27,3.7260


In [111]:
nav_month_end['year'] = nav_month_end['date'].dt.year
nav_month_end['month'] = nav_month_end['date'].dt.month

In [112]:
nav_month_end['monthly_return'] = (nav_month_end['nav_adjusted'] - nav_month_end['nav_adjusted'].shift(1)) / nav_month_end['nav_adjusted'].shift(1) * 100
nav_month_end

,date,nav_adjusted,year,month,monthly_return
4,2019-06-28,0.9997,2019,6,NaN
8,2019-07-26,0.9993,2019,7,-0.040012
13,2019-08-30,0.9463,2019,8,-5.303713
18,2019-09-30,0.9590,2019,9,1.342069
21,2019-10-25,0.9860,2019,10,2.815433
...,...,...,...,...,...
273,2024-09-30,3.2840,2024,9,31.391534
276,2024-10-25,3.3492,2024,10,1.985384
281,2024-11-29,3.6965,2024,11,10.369641
285,2024-12-27,3.7260,2024,12,0.798052


In [113]:
month_list = ['1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月']
year_list = nav_month_end["year"].unique()
month_return = pd.DataFrame(0,columns = month_list,index = year_list)
month_return

,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月
2019,0,0,0,0,0,0,0,0,0,0,0,0
2020,0,0,0,0,0,0,0,0,0,0,0,0
2021,0,0,0,0,0,0,0,0,0,0,0,0
2022,0,0,0,0,0,0,0,0,0,0,0,0
2023,0,0,0,0,0,0,0,0,0,0,0,0
2024,0,0,0,0,0,0,0,0,0,0,0,0
2025,0,0,0,0,0,0,0,0,0,0,0,0


In [114]:
for _, row in nav_month_end.iterrows():
    year = row["date"].year
    month_str = row["date"].month
    month_str = f"{month_str}月" 
    monthly_return_value = row["monthly_return"]
    month_return.loc[year, month_str] += monthly_return_value

In [115]:
month_return

,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月
2019,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,-0.040012,-5.303713,1.342069,2.815433,-3.640974,-0.905168
2020,2.039299,15.603206,23.446786,-0.175055,8.081251,3.806111,14.946272,-1.558074,-7.211511,-0.700906,3.285652,3.326278
2021,5.062921,8.724234,-0.148596,-4.485041,-1.348520,-2.575972,3.303706,-5.698052,14.844781,6.280604,0.286775,-2.057941
2022,3.321687,-3.321443,-6.257786,0.010223,-0.654196,4.151662,5.986663,-6.109894,-3.008041,-6.581372,-0.032870,5.819816
2023,6.499223,-1.998541,0.198472,9.185897,0.471677,-1.124001,5.341490,-0.667418,-1.221640,3.193463,-1.733510,-0.923425
2024,0.158270,-0.061452,12.069571,3.766264,-0.185066,-3.942788,-0.263925,-1.283621,31.391534,1.985384,10.369641,0.798052
2025,3.947933,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [25]:
def multi_fund_comparison(tables,fund_name):
    data1 = tables[0]
    data2 = tables[1].head(1)
    data3 = pd.DataFrame(columns=[2019,2020,2021,2022,2023,2024])
    data4 = tables[2]
    years = [2019, 2020, 2021, 2022, 2023, 2024]
    for year in years:
        data3[year] = data4.loc[data4["分年度业绩"] == year, f"{fund_name}_收益"].values
    data = pd.concat([data1,data2,data3], axis=1)
    data.drop(columns=["基准指数","整体业绩"], inplace=True)
    return data

In [31]:
basic_info = load_data("产品目录.xlsx")
data = pd.DataFrame()
files_list_series = pd.Series([i for i in Path("./data").rglob("*.xlsx")])
for row in basic_info.itertuples(index=False, name=None):
    nav_df_path = files_list_series[files_list_series.apply(lambda x: row[3] in x.stem)]
    assert len(nav_df_path) == 1, "找到多个文件"
    demo = NavResearch(nav_df_path.item(),row[0],row[3],row[4],row[5],row[6])
    demo.get_data()
    tables = demo.get_analysis_table()
    nav_df = multi_fund_comparison(tables,row[3])
    data = pd.concat([data,nav_df], axis=0)

草本优益1号无法推断频率,自动转为周度
济海实恒对冲二号无法推断频率,自动转为周度
均成CTA1号无法推断频率,自动转为周度
弘源多元化CTA无法推断频率,自动转为周度
宏锡量化CTA7号无法推断频率,自动转为周度
会世元丰CTA1号无法推断频率,自动转为周度
远澜翠柏1号无法推断频率,自动转为周度


In [33]:
data.to_excel('data.xlsx', index=False)